Names:

In [1]:
# your import statements


# WorkSheet Instructions ##

Before you begin you should have read and worked through Lab 5.

 I recommend that you do this worksheet in a python
  notebook and share screen.  This method does mean one person will do the typing.  When complete, print or Latex to pdf and upload to CANVAS.

This question is based on the adaptive timestep in Runge-Kutta section on Lab 5. The Runge-Kutta algorithm with adaptive time steps will, in general, be more efficient and accurate than the same algorithm with fixed time steps. In other words, greater accuracy can usually be achieved in fewer time steps.

  

For the given set of Daisyworld parameters and initial conditions in adapt_worksheet.yaml:

## Question A ###
Decrease the error tolerances for the adaptive timestep Runge-Kutta and compare the plots. Compare to the solutions for the algorithm with fixed timesteps. What do you see as the error tolerances are decreased?  You will note that as the error tolerances are decreased, the plots approach the one created by the algorithm with fixed time steps. What does this imply?


## Question B ###  
Compare the Daisyworld plot to a plot of the stepsizes. Do you see a correlation between stepsize and the shape of the curve?


## Question C ###  
Change the tolerances and see where you would set the tolerance to get (roughly) the same plot as the fixed timestep solution, but in the fewest time steps. Calculate the difference in timesteps between the fixed and adaptive solutions. Work out (by "hand") roughly how much computational time the adaptive timestep algorithm has saved with your chosen tolerance if the algorithm uses a. step doubling, and b. an embedded Runge-Kutta. What is the optimal tolerance value for both accuracy and time efficiency?